In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier  
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score  
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import itertools
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
np.set_printoptions(threshold=np.nan)

pdata = pd.read_csv("../Data/preprocessed_final.csv")
# pdata = pdata.drop(["Unnamed: 0"], axis=1)
pdata
pdata['Conditie'] = pdata['Conditie'].map({'norm': 0,  'bril': 1})
pdata = pdata.drop(pdata[pdata["Conditie"] > 0].index)
pdata = pdata.drop(pdata[pdata.Alcohol.eq(0)].sample(82).index)

pdata['Alcohol_Class'] = pdata['Alcohol'].astype(float)
conditions = [
    (pdata['Alcohol'] == 0),
    (pdata['Alcohol'] > 0) & (pdata['Alcohol'] < 0.5),
    (pdata['Alcohol'] > 0.5)]
choices = [0,1,2]
pdata['Alcohol_Class'] = np.select(conditions, choices, default=0)
pdata.groupby('Alcohol_Class').count()
# pdata
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(pdata)

,Conditie,SubjectID,Height,Weight,Age,BMI,Alcohol,Balance_MeanVelocity,GaitVelocity,MovementVelocity,std_SegmentData,mad_SegmentData,mean_StepLength,GaitTime
Alcohol_Class,,,,,,,,,,,,,,
0,8,8,8,8,8,8,8,8,8,8,8,8,8,8
1,28,28,28,28,28,28,28,28,28,28,28,28,28,28
2,36,36,36,36,36,36,36,36,36,36,36,36,36,36


In [ ]:
X = pdata.drop(['Conditie', 'SubjectID', 'Alcohol_Class', 'Alcohol'], axis=1)
Y = pdata['Alcohol_Class']
from pandas import read_csv
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
# feature extraction
model = LogisticRegression()
rfe = RFE(model, 2)
fit = rfe.fit(X, Y)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)
list(X)

In [ ]:
import seaborn as sns
corr = X.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True), square=True)

In [ ]:
#Splits data into training and validation according to specified ratio
def validation_split(data, ratio):
    sidlist = data['SubjectID'].unique()
    gdata = [data[data['SubjectID']==sid] for sid in sidlist]
    np.random.shuffle(gdata)
    N = len(gdata)
    training = gdata[:int(ratio*N)]
    validation = gdata[(int(ratio*N)):]
    return training,validation

# Converts the arrays in a list to one flattend dataframe
def flatten(data, variable_list):
    return pd.DataFrame(np.vstack(data), columns=variable_list)

def get_split_data(originaldata, variable_list, ratio):
    data = originaldata[variable_list]
    training,validation = validation_split(data, 0.7)
    training = flatten(training, variable_list)
    validation = flatten(validation, variable_list)

    X_train = training.drop(['Alcohol_Class', 'SubjectID'], axis=1)
    y_train = training['Alcohol_Class']
    X_train = X_train.astype(float)
    y_train = y_train.astype(int)
    
    X_test = validation.drop(['Alcohol_Class', 'SubjectID'], axis=1)
    y_test = validation['Alcohol_Class']
    X_test = X_test.astype(float)
    y_test = y_test.astype(int)
    return X_train, X_test, y_train, y_test

collist =  ['SubjectID', 'Alcohol_Class', 'GaitVelocity', 'MovementVelocity']
X_train, X_test, y_train, y_test = get_split_data(pdata, collist , 0.7)


X_train = np.array(X_train, dtype=float)
y_train = np.array(y_train, dtype=int)

X_test = np.array(X_test, dtype=float)
y_test = np.array(y_test, dtype=int)



In [ ]:
variable_list = ['SubjectID', 'Alcohol_Class', 'GaitVelocity', 'MovementVelocity']

data = pdata[variable_list]
training,validation = validation_split(data, 0.7)
training = flatten(training, variable_list)
validation = flatten(validation, variable_list)

X_train = training.drop(['Alcohol_Class', 'SubjectID'], axis=1)
y_train = training['Alcohol_Class']
X_train = X_train.astype(float)
y_train = y_train.astype(int)

X_test = validation.drop(['Alcohol_Class', 'SubjectID'], axis=1)
y_test = validation['Alcohol_Class']
X_test = X_test.astype(float)
y_test = y_test.astype(int)

X_train = np.array(X_train, dtype=float)
y_train = np.array(y_train, dtype=int)

X_test = np.array(X_test, dtype=float)
y_test = np.array(y_test, dtype=int)
    
classifier = DecisionTreeClassifier(max_depth=5, min_samples_leaf=0.01)  
classifier.fit(X_train, y_train)  
y_pred = classifier.predict(X_test) 

accuracy = accuracy_score(y_test, y_pred)*100
print('Accuracy of our model is equal ' + str(round(accuracy, 2)) + ' %.')


In [ ]:
variable_list = ['SubjectID', 'Alcohol_Class', 'GaitVelocity', 'MovementVelocity']

def make_alcohol_decisions(originaldata, variable_list):
    data = originaldata[variable_list]
    training,validation = validation_split(data, 0.7)
    training = flatten(training, variable_list)
    validation = flatten(validation, variable_list)

    X_train = training.drop(['Alcohol_Class', 'SubjectID'], axis=1)
    y_train = training['Alcohol_Class']
    X_train = X_train.astype(float)
    y_train = y_train.astype(int)
    
    X_test = validation.drop(['Alcohol_Class', 'SubjectID'], axis=1)
    y_test = validation['Alcohol_Class']
    X_test = X_test.astype(float)
    y_test = y_test.astype(int)
    
#     scaler = StandardScaler()

#     # Fit on training set only.
#     scaler.fit(X_train)

#     # Apply transform to both the training set and the test set.
#     X_train = scaler.transform(X_train)
#     X_test = scaler.transform(X_test)
    
#     lda = LDA(n_components=1)  
#     X_train = lda.fit_transform(X_train, y_train)  
#     X_test = lda.transform(X_test) 
    
    classifier = DecisionTreeClassifier(max_depth=5, min_samples_leaf=0.01)  
    classifier.fit(X_train, y_train)  
    y_pred = classifier.predict(X_test) 
    return accuracy_score(y_test, y_pred)*100


make_alcohol_decisions(pdata, variable_list)

In [ ]:
print('mad_SegmentData, mean_StepLength:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class','mean_StepLength', 'mad_SegmentData']) for i in range(100)]), 2),'%.')
print('mean_StepLength, std_SegmentData:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'mean_StepLength', 'std_SegmentData']) for i in range(100)]), 2),'%.')
print('GaitTime, mean_StepLength, mad_SegmentData:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'mean_StepLength', 'GaitTime', 'mad_SegmentData']) for i in range(100)]), 2),'%.')
print('std_SegmentData, GaitVelocity, MovementVelocity, mean_StepLength:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'GaitVelocity', 'MovementVelocity', 'mean_StepLength', 'std_SegmentData']) for i in range(100)]), 2),'%.')
print('GaitTime, GaitVelocity, MovementVelocity, mean_StepLength, std_SegmentData:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'GaitVelocity', 'MovementVelocity', 'mean_StepLength', 'GaitTime', 'std_SegmentData']) for i in range(100)]), 2),'%.')
print('GaitTime, mean_StepLength, Height:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'mean_StepLength', 'GaitTime', 'Height']) for i in range(100)]), 2),'%.')
print('GaitTime, mean_StepLength, Age:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'mean_StepLength', 'GaitTime', 'Age']) for i in range(100)]), 2),'%.')
print('mean_StepLength, Height:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'mean_StepLength', 'Height']) for i in range(100)]), 2),'%.')
print('mean_StepLength, Age:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'mean_StepLength', 'Age']) for i in range(100)]), 2),'%.')
print('GaitTime, Height:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'GaitTime', 'Height']) for i in range(100)]), 2),'%.')
print('std_SegmentData, Height:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'mad_SegmentData', 'Height']) for i in range(100)]), 2),'%.')
print('mad_SegmentData, Age:', round(np.mean([make_alcohol_decisions(pdata, ['SubjectID', 'Alcohol_Class', 'mad_SegmentData', 'Age']) for i in range(100)]), 2),'%.')


more equal
mad_SegmentData, mean_StepLength: 52.73 %.
mean_StepLength, std_SegmentData: 52.39 %.
GaitTime, mean_StepLength, mad_SegmentData: 54.12 %.
std_SegmentData, GaitVelocity, MovementVelocity, mean_StepLength: 52.35 %.
GaitTime, GaitVelocity, MovementVelocity, mean_StepLength, std_SegmentData: 54.46 %.
GaitTime, mean_StepLength, Height: 50.45 %.
GaitTime, mean_StepLength, Age: 52.09 %.
mean_StepLength, Height: 51.88 %.
mean_StepLength, Age: 51.36 %.
GaitTime, Height: 50.96 %.
std_SegmentData, Height: 49.68 %.
mad_SegmentData, Age: 48.62 %.


In [ ]:
from sklearn import neighbors, datasets
from matplotlib.colors import ListedColormap
# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00'])

h = .02  # step size in the mesh
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, m_max]x[y_min, y_max].
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])
colors = {0: 'red', 1: 'blue'}
# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
# plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cmap_bold)

score = logreg.score(X_test, y_test)
# plt.scatter(X_test[:, 0], X_test[:, 1], c=y_pred, alpha=0.5, cmap=cmap_bold)

for label in np.unique(y_test):
    indices = np.where(y_test == label)
    plt.scatter(X_test[indices, 0], X_test[indices, 1], c=colors[label], alpha=0.8, 
                label=' {}'.format(label))

colors = ['red', 'blue']

labels = ['0', '1']
plt.legend(labels)
plt.title('Logistic regression')
plt.xlabel('Step length')
plt.ylabel('Sum distance')
plt.savefig('Logistic regression alcohol')
plt.show()